# Qubitization methodology for simulating Coupled Classical Harmonic Oscillators using Classiq
Authors: Viraj Dsouza, Cristina Radian, Kerem Yurtseven

Date: 09.08.2024

This work is designed under and for the final project of the Womanium Quantum+AI Program 2024

Reference Paper: Exponential Quantum Speedup in Simulating Coupled Classical Oscillators, Babbush et.al, 2023 [1](https://arxiv.org/abs/2303.13012)

We also refer the readers to [Clasiqdocs](https://github.com/Classiq/classiq-library/tree/9c43f05f3d498c8c72be7dcb3ecdaba85d9abd6e/tutorials/hamiltonian_simulation/hamiltonian_simulation_with_block_encoding) for the qubitization methodology and use this for the purpose of our work.
___


 


## Qubitization

Our purpose in this notebook is to block encode the Hamiltonian evolution matrix such that:
\begin{equation*}
    U = \begin{pmatrix} \text{exp}(iHt)/ \lambda & * \\
    * & *
\end{pmatrix}
\end{equation*}
where $H$ is the system Hamiltonian and $\lambda$ is the normalization factor and use that unitary matrix to create a random walk operator and simulate the behavior of the system at a given $t$.
\begin{equation*}
    \ket{\psi(t)} = e^{-iHt} \ket{\psi(0)}
\end{equation*}

In order to create a block encoding, we first imply LCU method to the system Hamiltonian to first form a block encoding of $H$.
___

## Linear Combination of Unitaries

Assume $H$ can be written in Pauli basis as:
\begin{equation*}
    H = \sum_{i=0}^{L-1} \alpha_i U_i
\end{equation*}

LCU refers to $(\bar{\alpha}, m, 0)$-block encoding of a matrix given as a sum of unitaries:
$$
U_{(\bar{\alpha},m,0)-H} =\begin{pmatrix}
H/\bar{\alpha} & * \\
* & *
\end{pmatrix}, \text{   for     } H = \sum^{L-1}_{i=0} \alpha_i U_i, \,\,\,\,\, \alpha_i\geq 0
$$
with $\bar{\alpha}\equiv\sum^{L-1}_{i=0}\alpha_i$ and $m= \lceil\log_2(L) \rceil$ represents the numner of block qubits. 


More details on the LCU methodology can be found in classiq's [LCU tutorial](https://github.com/Classiq/classiq-library/blob/main/tutorials/linear_combination_of_unitaries/linear_combination_of_unitaries.ipynb)

We can define the normalization as $ \lambda = \sum |\alpha_i|$ and find the `PREPARE` and `SELECT` functions.
___ 

## Quantum Walk-Operator and block encoding $e^{iHt}$
To simulate the Hamiltonian, we can use the Szegedy Quantum Walk-Operator [2](https://ieeexplore.ieee.org/abstract/document/1366222).




Given block-encoding of the Hamiltonian $H$, we can define the Szegedy quanutm walk-operator as: 
$$
W\equiv -\Pi_{|0\rangle_m} U_{(\bar{\alpha},m,0)-H},
$$  
where $\Pi_{|0\rangle_m}$ is a reflection operator about the block state. The powers of $W$ correspond to a $(1,m,0)$ block encoding of the Chebyshev polynomials of the matrix $H$:
$$
W^k = \begin{pmatrix}
T_k(H) & * \\
* & *
\end{pmatrix}=U_{(1,m,0)-T_k(H)},
$$
with $T_k$ being the k-th Chebyshev polynomial. Its useful to note that this property holds only for an **Hermitian block-encoding**. From the Jacobi -Anger expansion (given below) we can have an $\epsilon$-approximation of $\exp(iHt)\approx \sum^d_{i=0} \beta_{i} T_{i}(H)$, for which we can perform the following encoding
$$
U_{(\bar{\beta},\tilde{m},\epsilon)-\exp{(iHt)}} =
\begin{pmatrix}
\exp{(iHt)}/\bar{\beta} & * \\
* & *
\end{pmatrix}=
% \begin{pmatrix}
% \frac{1}{\bar{\beta}}\sum^d_{k=0} \beta_{k} T_{k}(Ht) & * \\
% * & *
% \end{pmatrix}=
\begin{pmatrix}
\frac{1}{\bar{\beta}}\sum^d_{k=0} \beta_{k} W^k & * \\
* & *
\end{pmatrix}
% \begin{pmatrix}
% \frac{1}{\bar{\beta}}\sum^d_{k=0} W^k & * \\
% * & *
% \end{pmatrix},
$$
where $\tilde{m}=m+\lceil \log_2(d+1) \rceil$ 

We evaluate the coefficients $\beta_{i}$ as the bessel coefficients ($J(t)$) of the Jacobi-anger expressions for sine and cosine as follows
$$
\cos(Ht)= J_0(t) + 2\sum^{d}_{k=1} (-1)^k J_{2k}(t) T_{2k}(H)\\
\sin(Ht)= 2\sum^{d}_{k=0} (-1)^k J_{2k+1}(t) T_{2k+1}(H),
$$
___

##  System Configuration
In this example, let exist two masses with $m_1 = 1$ and $m_2 = 1$. Also, assume spring constants are $\kappa_{11} = \kappa_{22} = 0$ and $\kappa_{12} = 1$. Initial states are chosen as $\vec{x}(t) = (0,1)^T$ and $\vec{\dot{x}}(t) = (0,0)^T$

![Toy Example](Figures/Toy_Example.png)

___

## Pre-Processing

Create necessary matrix and initial states using functions created in `ccho_helpers.py`

In [1]:
## Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy
from ccho_helpers import *
from classiq import * 
from classiq.execution import ExecutionPreferences, ClassiqBackendPreferences, IBMBackendPreferences, IBMBackendProvider, IonqBackendPreferences, GCPBackendPreferences

In [2]:
## Creating the 2 masses system

# Number of masses
N =2**1

# ass Matrix
M = np.array([[1,0],[0,1]])

# Spring Matrix
K = np.array([[0,1],[1,0]])

# Initial Conditions
x_0 = np.zeros([N,1])
xdot_0 = np.zeros([N,1])
x_0[1] = 1

# F Matrix
F= create_matrix_F(K, N)

# A Matrix
A = create_matrix_A(M, F)

# Transform coordinates
y = coordinate_transformation(M, x_0, xdot_0)
y_0 = y["y_0"]
ydot_0 = y["ydot_0"]

# B Matrix
B = create_matrix_B(M, K, A, N)

# Stacking B with zeros to get square matrix N^2 x N^2
B_padded = padding_B(B, N)

# Hamiltonian
H = create_Hamiltonian(B_padded)

# Initial State
init_state = create_init_state(B_padded, y_0, ydot_0, N)
E0_y = calculate_energy(y_0, ydot_0, M, K)

# Normalize the Initial State
normalization = normalize_init_state(init_state)
normalized_init_state = normalization["normalized_init_state"]
norm = normalization["norm"]

# Print the results
print("Initial Energy: ", E0_y)
print("B matrix: ", B)
print("Hamiltonian formed:", H)
print("Initial State:", init_state)

Initial Energy:  0.5
B matrix:  [[ 0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j]]
Hamiltonian formed: [[-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -1.-0.j -0.-0.j -0.-0.j]
 [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j  1.-0.j -0.-0.j -0.-0.j]
 [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-1.+0.j  1.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]]
Initial State: [[ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [-0.-1.j]
 [ 0.+0.j]
 [ 0.+0.j]]


## Pauli Matrix Decomposition

Create the Pauli List for the Hamiltonian. Pauli Decomposition is taken from [Classiq Library](https://github.com/Classiq/classiq-library/blob/main/algorithms/hhl/hhl/hhl.ipynb) thanks a lot to the authors!

In [3]:
## Pauli Decomposition and transforming it into Classiq syntax

# Create the Pauli Matrix Decomposition of the Hamiltonian
pauli_list = lcu_naive(H)

# Transform Pauli Matrix Decomposition to Classiq compatible
classiq_pauli_list = pauli_list_to_hamiltonian(pauli_list)

print(pauli_list)

100%|██████████| 64/64 [00:00<00:00, 32936.87it/s]

[('XII', (0.25+0j)), ('XIZ', (-0.25+0j)), ('XIX', (-0.25+0j)), ('XZI', (0.25+0j)), ('XZZ', (-0.25+0j)), ('XZX', (-0.25+0j)), ('YIY', (0.25+0j)), ('YZY', (0.25+0j))]


## LCU and Chebyshev Functions

Create functions to find LCU probabilities and Chebyshev coefficients. These functions are taken from the Classiq Github [4](https://github.com/Classiq/classiq-library/blob/main/tutorials/hamiltonian_simulation/hamiltonian_simulation_with_block_encoding/hamiltonian_simulation_with_block_encoding.ipynb). Thanks a lot to the authors!

In [4]:
def get_cheb_coef(epsilon, t):
    """
        Definition:
            Find the Chebyshev coefficients
        Args:
            epsilon (float): Sensitivity
            t (float): Evolution time for the system
        Outputs:
            cos_coef (list): Coefficients of cosine function
            sin_coef (list): Coefficients of sine function
    """
    poly_degree = int(
        np.ceil(
            t
            + np.log(epsilon ** (-1)) / np.log(np.exp(1) + np.log(epsilon ** (-1)) / t)
        )
    )
    cos_coef = [scipy.special.jv(0, t)] + [
        2 * scipy.special.jv(2 * k, t) * (-1) ** k for k in range(1, poly_degree // 2 + 1)
    ]
    sin_coef = [
        -2 * scipy.special.jv(2 * k - 1, t) * (-1) ** k for k in range(1, poly_degree // 2 + 1)
    ]
    return cos_coef, sin_coef

#### **Block Encoding the Hamiltonian (LCU):**

##### Preparing the normalized coefficients for the PREPARE operation

The PREPARE operation prepares a quantum state that corresponds to the probabilities $|\alpha_i| /\sum|\alpha_i|$. We define a function that gets the list of  coefficients of the Hamiltonian and returns the probabilities to be loaded as part of the PREPARE operation. The block size is also determined based on the number of terms in the Hamiltonian.

In [5]:
def get_normalized_lcu_coef(lcu_coef):
    """
        Definition:
            Find the LCU probabilities, normaliziation factor, and size of the probabilities
        Args:
            lcu_coef (list): List of coefficients for Pauli matrices
        Outputs:
            normalization_factor (float): 1-norm of the coefficients
            coef_size (int): Number of Qubits to create the probabilities
            prepare_prob(list): Probabilities of normalized LCU coefficients
    """
    
    lcu_coef = [abs(c) for c in lcu_coef]
    normalization_factor = sum(lcu_coef)
    prepare_prob = [c / normalization_factor for c in lcu_coef]
    coef_size = int(np.ceil(np.log2(len(prepare_prob))))
    prepare_prob += [0] * (2**coef_size - len(prepare_prob))

    return normalization_factor, coef_size, prepare_prob

#### Defining a function that applies the $(PREPARE) SELECT (PREPARE^{\dagger})$ operation

The SELECT operation acts on the desired target state controlled by the quantum state of the block qubits. This constitution ends our construction of the unitary block Hamiltonian. 

In [6]:
@qfunc
def apply_pauli_term(pauli_string: PauliTerm, state: QArray[QBit]):
    """
    Definition:
        Apply the Pauli Matrix to the state
    Args:
        pauli_string (PauliTerm): Pauli Matrix sequence
        state (QArray[QBit]): State to apply pauli matrices
    """
    repeat(
        count=state.len,
        iteration=lambda index: switch(
            pauli_string.pauli[index],
            [
                lambda: IDENTITY(state[pauli_string.pauli.len - index - 1]),
                lambda: X(state[pauli_string.pauli.len - index - 1]),
                lambda: Y(state[pauli_string.pauli.len - index - 1]),
                lambda: Z(state[pauli_string.pauli.len - index - 1]),
            ],
        ),
    )


@qfunc
def lcu_paulis(pauli_terms_list: CArray[PauliTerm], signs: CArray[CInt], probs: CArray[CReal], block: QNum, state: QArray[QBit]):
    """
        Definition:
            Apply all the Pauli sequences in LCU
        Args:
            pauli_terms_list (CArray[PauliTerm]): Pauli Matrix Decomposition of the Hamiltonian
            signs (CArray[CInt]): Signs of PMD coefficients
            probs (CArray[CReal]): Probabilities for LCU
            block (QNum): PREPARE variables
            state (QArray[QBit]): State to apply LCU
    """

    within_apply(
        lambda: inplace_prepare_state(probs, 0.0, block),
        lambda: repeat(
            count=pauli_terms_list.len,
            iteration=lambda i: control(
                block == i, lambda: (
                                U(0, 0, 0, np.pi / 2 * signs[i], state[0]),
                                apply_pauli_term(pauli_terms_list[i], state)
                ),
            ),
        )
    )

In [7]:
## Find the LCU parameters for the Hamiltonian

pauli_coeff = [p.coefficient for p in classiq_pauli_list]
signs_pauli = np.sign(pauli_coeff).tolist()
generalized_signs_pauli = [
    (1 - signs_pauli[s]) + (s) % 2 for s in range(len(signs_pauli))
]
normalization_ham, lcu_size_ham, prepare_probs_ham = get_normalized_lcu_coef(pauli_coeff)

print("Normalization factor: ", normalization_ham)
print("LCU size: ", lcu_size_ham)
print("Probabilities: ", prepare_probs_ham)
print("Signs: ", generalized_signs_pauli)

Normalization factor:  2.0
LCU size:  3
Probabilities:  [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
Signs:  [0.0, 3.0, 2.0, 1.0, 2.0, 3.0, 0.0, 1.0]


## Hamiltonian Simulation using Block Encoding $e^{iHt}$: (Qubitization method)
Define necessary functions for Quantum Walk-Operators and Qubitization and simulate the Hamiltonian

In [8]:
@qfunc
def my_walk_operator(block: QArray[QBit], state: QArray[QBit]) -> None:
    """
        Definition:
            Applies the Szegedy Quantum Walk Operator
        Args:
            block (QArray[QBit]): PREPARE variables
            state (QArray[QBit]): State to apply walk operator
    """
    lcu_paulis(classiq_pauli_list, generalized_signs_pauli, prepare_probs_ham, block, state)
    reflect_about_zero(block)
    RY(2 * np.pi, block[0])

##### Evaluating the Chebyshev co-efficients

We calculate the coefficients $\beta_i$ for approximating the sine and cosine functions. We want to approximate $e^{iHt}$, but our Hamiltonian is encoded with a normalization factor $\bar{\alpha}$ that comes from the LCU block encoding. Therefore, we rescale the time by this factor. 

Given the coefficients, we calculate the probabilities for the LCU prepare variable. These must be positive numbers. Moreover, for the odd terms, which corresponds to the sine function, we should add a factor of $i$. Those factors should be added to the unitary operations $W^k$. To take this into account, for each term we define a "generalized sign" $\sigma_k$ such that $\beta_k = e^{\frac{\pi}{2}\sigma_k i} |\beta_k|$ with $\sigma_k \in \{0,1,2,3\}$ that corresponds to the factors $1,i,-1,-i$ respectively. 

In [9]:
## Finc Chebyshev coefficients and take phases into account

time = 0.5

normalized_time = normalization_ham * time

cos_coef, sin_coef = get_cheb_coef(0.0001, normalized_time)

combined_sin_cos_coef = []
for k in range(len(cos_coef) - 1):
    combined_sin_cos_coef.append(cos_coef[k])
    combined_sin_cos_coef.append(sin_coef[k])
combined_sin_cos_coef.append(cos_coef[-1])
if len(sin_coef) == len(cos_coef):
    combined_sin_cos_coef.append(sin_coef[-1])

signs_cheb_coef = np.sign(combined_sin_cos_coef).tolist()
generalized_signs = [
    (1 - signs_cheb_coef[s]) + (s) % 2 for s in range(len(signs_cheb_coef))
]
positive_cheb_lcu_coef = np.abs(combined_sin_cos_coef)

In [10]:
## Find the LCU parameters for the exponentiation

normalization_exp, lcu_size_exp, prepare_probs_exp = get_normalized_lcu_coef(positive_cheb_lcu_coef)

print("Normalization factor: ", normalization_exp)
print("LCU size: ", lcu_size_exp)
print("Probabilities: ", prepare_probs_exp)

Normalization factor:  1.9191858138051914
LCU size:  3
Probabilities:  [0.3987095366450216, 0.4585804903095238, 0.11974190732900435, 0.02038713900649352, 0.0025809267099567547, 0, 0, 0]


In [11]:
## LCU function for Chebyshev coefficients

@qfunc
def lcu_cheb(coef: CArray[CReal], generalized_signs: CArray[CInt], walk_operator: QCallable[QNum, QArray], walk_block: QNum, state: QArray, cheb_block: QNum):
    """
        Definition:
            Apply all the Chebyshev coefficients in LCU
        Args:
            coef (CArray[CReal]): Chebyshev Coefficients
            generalized_signs (CArray[CInt]): Signs of Chebyshev coefficients
            walk_operator (QCallable[QNum, QArray]): Quantum Walk-Operator
            walk_block (QNum): PREPARE variables for Quantum Walk
            state (QArray[QBit]): State to apply LCU
            cheb_block (QNum): PREPARE variables for Chebyshev coefficients
    """

    within_apply(
        lambda: inplace_prepare_state(coef, 0.0, cheb_block),
        lambda: repeat(
            generalized_signs.len,
            lambda k: control(
                cheb_block == k,
                lambda: (
                    U(0, 0, 0, np.pi / 2 * generalized_signs[k], state[0]),
                    power(k, lambda: walk_operator(walk_block, state)),
                ),
            ),
        ),
    )

In [12]:
## Main Evolution

# Add i phase for the last half
@qfunc
def init_state_phase(state: QNum):
    """
        Definition:
            Imply -pi/2 phase to the last half (msb qubit) of the QNum state in order to add -i factor
        Args:
            state (QNum): Initial state with bare amplitudes
        Outputs:
            state (QNum): Initial state with phase
    """

    state_in_qubit = QArray("state_in_qubit")
    msb = QArray("msb", QBit)
    
    size = np.log2(init_state.size)
    allocate(size, msb)

    bind(state, state_in_qubit)
    repeat(state_in_qubit.len, lambda i: CX(state_in_qubit[i], msb[i]))
    control(msb[size-1], lambda: PHASE(np.pi/2, state_in_qubit[size-1]))
    bind(state_in_qubit, state)

# Main Function
@qfunc
def main(ham_block: Output[QNum], state: Output[QNum], exp_block: Output[QNum]):
    """
        Definition:
            Main function to make evolution

        Outputs:
            ham_block (Output[QNum]): PREPARE variables for Hamiltonian
            state (Output[QNum]): Resulting state at the t = evolution_coefficient
            exp_block (Output[QNum]): PREPARE variables for exponantioation
    """

    allocate(lcu_size_exp, exp_block)
    allocate(lcu_size_ham, ham_block)
    prepare_amplitudes(list(normalized_init_state), 0.0, state)
    init_state_phase(state)

    lcu_cheb(
        prepare_probs_exp,
        generalized_signs,
        lambda x, y: my_walk_operator(x, y),
        ham_block,
        state,
        exp_block,
    )


qmod = create_model(main)
write_qmod(qmod, "Simulation with Qubitization", decimal_precision=16)

In [14]:
# Create model and synthesis. Use "simulator_statevector" to take phases into account
backend_preferences = ClassiqBackendPreferences(backend_name="simulator_statevector")
model_pref = set_execution_preferences(qmod, ExecutionPreferences(num_shots=1, backend_preferences=backend_preferences))
qprog = synthesize(model_pref)
show(qprog)

Opening: https://platform.classiq.io/circuit/d015725d-04d8-4de6-9545-d7ffe0317e91?version=0.43.3


In [15]:
# Execute the job
job = execute(qprog)
results = job.result()

## Post-Processing and Comparision

In [16]:
def get_projected_state_vector(execution_result, measured_var: str, projections: dict) -> np.ndarray:
    """
    Definition:
        This function returns a reduced statevector from execution results.
        measured var: the name of the reduced variable
        projections: on which values of the other variables to project, e.g., {"ind": 1}
    """
    
    projected_size = len(execution_result[0].value.output_qubits_map[measured_var])
    proj_statevector = np.zeros(2**projected_size).astype(complex)
    for sample in execution_result[0].value.parsed_state_vector:
        if all(sample.state[key] == projections[key] for key in projections.keys()):
            proj_statevector[int(sample.state[measured_var])] += sample.amplitude
    return proj_statevector

In [17]:
## Get the states where blocks are in 0 state

state_result = get_projected_state_vector(
    results, "state", {"exp_block": 0.0, "ham_block": 0.0}
)

In [18]:
## Renormalize the final result with the norm of the initial state
final_state_result = norm * state_result

In [19]:
## Compare with classical results
expected_state = (1 / normalization_exp * scipy.linalg.expm(1j * H * time) @ init_state)

relative_phase = np.angle(expected_state[0] / state_result[0])

final_state_result = final_state_result * np.exp(1j * relative_phase)

print("Expected State:", expected_state)
print("Resulting State:", final_state_result)


Expected State: [[-0.2393529+0.j        ]
 [ 0.2393529+0.j        ]
 [ 0.       +0.j        ]
 [ 0.       +0.j        ]
 [ 0.       +0.j        ]
 [ 0.       -0.39612871j]
 [ 0.       +0.j        ]
 [ 0.       +0.j        ]]
Resulting State: [-1.62132687e-01+3.54829846e-17j  1.62132687e-01-1.83459399e-15j
  3.80342924e-17-5.92531917e-17j  5.66967202e-17+3.52190715e-17j
 -1.61220529e-15-3.33056534e-15j -3.23352879e-01-2.38682564e-01j
  5.33793253e-17+7.24025475e-17j  7.85852506e-17+2.81873394e-17j]


In [20]:
## Find the overlap between two result

overlap = np.abs(np.vdot(state_result, expected_state)) * normalization_exp/ np.linalg.norm(state_result)
print("Overlap:", overlap)

Overlap: 0.890038098309116


## Discussion and Results

##### This work shows that Qubitization and Quantum Walk-Operators are important tools in Hamiltonian evolutions. We see a 89% overlap between classical and quantum results and this can be imrpoved by decreasing the epsilon for Chebyshev coefficients. Moreover the codes are general and can be used for simulating for larger values of $N=2**n$

___
___